# **All The Words**
An old school multiplayer word game.

Each turn the player gets 10 random letters and 1 minute to enter as many words as he can build using the 10 letters or some of them.
The score of each turn is calculated by the amount of valid words and their length.

1 letter words are worth 1 point, 2 letter words are worth 2 points, 3 letter words are worth 3 points .... etc.
And 5 additional points for longest word. 

At the end of the game we display the winner and the longest word.

The 1 min turn may be tricky to implement as we did not learn async methods... Maybe some logic that does not stops the player after 1 min but checks the time after entering each word and stops if the time is more then 1 min - this approach does not require async methods. 
https://pypi.org/project/pyenchant/

# Setup the environment 

In [45]:
!apt -qq install enchant
!pip install pyenchant
!pip install inputimeout
import sys
import random
import string
import selectors
import termios

import enchant
from datetime import datetime

TIMEOUT = 20
VOWELS = "aeiou"
CONSONANTS = "".join(set(string.ascii_lowercase) - set(VOWELS))

enchant is already the newest version (1.6.0-11.1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


# Define entities

In [46]:
class Player:
    def __init__(self, player_name):
        self.name = player_name
        self.winning_rounds = 0
        self.longest_word = None

class Round:
  def __init__(self):
    self.random_letters = self.random_letter_selection()
    self.distinct_words_entered = []
    self.dict_invalid_words = []
    self.game_invalid_words = []
    self.valid_words = []
    self.score = 0
    self.longest_word = None
  
  def random_letter_selection(self):
    return [random.choice(VOWELS) for i in range(0,3)] + [random.choice(CONSONANTS) for i in range(0,7)]

  def __str__(self):
    round_status = []
    round_status.extend(["\nWords entered : " + ",".join(self.distinct_words_entered) if self.distinct_words_entered != None and len(self.distinct_words_entered) > 0 else ""])
    round_status.extend(["\nWords not in EN dictionary : " + ",".join(self.dict_invalid_words) if self.dict_invalid_words != None and len(self.dict_invalid_words) > 0 else ""])
    round_status.extend(["\nWords with invalid letters : " + ",".join(self.game_invalid_words) if self.game_invalid_words != None and len(self.game_invalid_words) > 0 else ""])
    round_status.extend(["\nValid words : " + ",".join(self.valid_words) if self.valid_words != None and len(self.valid_words) > 0 else "\nNo valid words!"])
    round_status.extend(["\nScore : " + str(self.score)])
    round_status.extend(["\nLongest words : " + ",".join(self.longest_word) if self.longest_word != None and len(self.longest_word) > 0 else ""])
    status_str = "".join(round_status)
    return status_str
    
  def score_round(self):
    word_dict = {}
    for i in range(1,10):
      word_list = [word for word in self.valid_words if len(word) == i]
      if word_list != None:
        word_dict[i] = word_list

    self.score = sum([key*len(value) for key,value in word_dict.items()])    

    for key in range(max(word_dict.keys()),1,-1):
      if len(word_dict[key]) > 0:
        self.longest_word =  word_dict[key]
        break

class WordGame:
  def __init__(self, players):
    self.en_dictionary = enchant.Dict("en_US")
    self.num_players = len(players)
    self.players =  [Player(name) for name in players]
    self.rounds = [Round() for name in players]    

  def get_player_answer(self, player_name, random_letters):
    answer = []
    print("{} use the following letters : {}\n to enter as many words are you can over the next {} seconds ...\n".format(player_name, random_letters, TIMEOUT))
    start_time = datetime.now()
    while ((datetime.now() - start_time).seconds < TIMEOUT):
      word = input(">>")
      if (datetime.now() - start_time).seconds < TIMEOUT:
        clean_word = word.strip()
        if len(clean_word) > 0 and clean_word not in answer:
          answer.append(clean_word)
    return answer

  def filter_invalid_words(self, answer, curr_round):

    curr_round.distinct_words_entered.extend(answer)
    curr_round.dict_invalid_words = [word for word in curr_round.distinct_words_entered if not self.en_dictionary.check(word)]

    curr_round_dict = dict((x,curr_round.random_letters.count(x)) for x in sorted(set(curr_round.random_letters)))

    for word in curr_round.distinct_words_entered:
      if word not in curr_round.dict_invalid_words:
        is_word_valid = True
        word_dict = dict((x,word.count(x)) for x in sorted(set(word)))

        for item in word_dict.keys():
          if item not in curr_round_dict.keys():
            is_word_valid = False
          elif word_dict[item] > curr_round_dict[item]:
            is_word_valid = False

        if is_word_valid:
          curr_round.valid_words.append(word)
        else:
          curr_round.game_invalid_words.append(word)
            
  def play_round(self, player_number):
    curr_player = self.players[player_number]
    curr_round = self.rounds[player_number]

    print("\n============================================================")
    answer = self.get_player_answer(curr_player.name, curr_round.random_letters)
    self.filter_invalid_words(answer, curr_round)
    curr_round.score_round()
    print(curr_round)

  def calc_results(self):
    print("\n============================================================")
    longest_word = {}
    max_word_len_game = 0

    for i in range(0,self.num_players):
      if self.rounds[i].longest_word != None:
        longest_word[i] = len(self.rounds[i].longest_word[0])

    if len(longest_word.values()) > 0:
      max_word_len_game = max(longest_word.values()) 
      for player_num, max_word_len in longest_word.items():
        if max_word_len == max_word_len_game:
          print("Congratulations {} you've entered the longest word, you get an extra 5 points".format(self.players[player_num].name))
          self.rounds[player_num].score += 5

    highest_score = 0
    winning_players = []
    for i in range(0,self.num_players):
      if self.rounds[i].score > highest_score:
        highest_score = self.rounds[i].score
        winning_players.clear()
        winning_players.append(self.players[i].name)
      elif self.rounds[i].score == highest_score:
        winning_players.extend(self.players[i].name)

    if highest_score == 0:
      print("No winners here :(")
    elif len(winning_players) == 1:
      print("#### And the winner is: {} ####".format(winning_players[0]))
    else:
      print("#### And the winners are: {} ####".format(",".join(winning_players)))


#Play Game

In [48]:
print("Welcome to All The Words. the laziest word game ever developed in Python")

player_names = input("Please enter the names of all the players, seperated by a comma\n")
valid_player_names =  [name.strip() for name in player_names.split(",") if len(name.strip()) > 0]
print("Good luck : ", ", ".join(valid_player_names))

game_on = WordGame(valid_player_names)

for i in range (0, game_on.num_players):
  game_on.play_round(i)

game_on.calc_results()





Welcome to All The Words. the laziest word game ever developed in Python


KeyboardInterrupt: ignored